In [2]:
def get_Level_or_Letter(is_level):
    gamer_input = input()
    if len(gamer_input) == 1:
        if is_level and gamer_input.isdigit():
            return gamer_input
        elif not is_level and gamer_input.isalpha():
            return gamer_input.lower()
    print('Error, try again')
    return get_Level_or_Letter(is_level)

In [3]:
class Letter():
    def __init__(self, letter, open_flag):
        self.letter = letter
        self.open_flag = open_flag
    
    def show(self):
        if self.open_flag:
            return self.letter
        else:
            return '*'

In [4]:
class Slovo():
    def __init__(self, word, attemts_number, is_letter_in_word=False):
        self.word = word
        self.attemts_number = attemts_number
        self.is_letter_in_word = is_letter_in_word

        letter_list = []
        for i in word:
            letter_list.append(Letter(i, False))
        self.letter_list = letter_list

        inputs_list = []
        self.inputs_list = inputs_list

    def check(self, input_letter):
        if input_letter not in self.inputs_list:
            self.inputs_list.append(input_letter)
        else:
            print(f'You have already entered the letter "{input_letter}"')
            self.is_letter_in_word = True
        
    def decrease_attemts(self, input_letter):
        if not self.is_letter_in_word:
            print(f'The letter "{input_letter}" is not in the word')
            self.attemts_number -=1

    def show(self, show_word_flag, input_letter=None):
        if show_word_flag:
            new_word = self.word
            return new_word
        else:
            new_word = ''
            self.is_letter_in_word = False
            for i in self.letter_list:
                if i.letter == input_letter:
                    self.is_letter_in_word = True
                    i.open_flag = True
                new_word += i.show()
            return new_word

    def game_mechanics(self, show_word_flag, input_letter):
        new_word = self.show(show_word_flag, input_letter)
        self.check(input_letter)
        self.decrease_attemts(input_letter)
                
        return (new_word, self.attemts_number)
            


In [5]:
def edit_level_sentence(dict, level):
    return f'\n{dict[str(level)][0]} - The word consists of {dict[str(level)][1]} letters, \
the number of attempts is {dict[str(level)][2]}'

In [6]:
def game(levels_dict):
    print(f'Select difficulty level:\
        {edit_level_sentence(levels_dict, 1)} (Press 1)\
        {edit_level_sentence(levels_dict, 2)} (Press 2)\
        {edit_level_sentence(levels_dict, 3)} (Press 3)')

    gamer_input = get_Level_or_Letter(True)
    print(edit_level_sentence(levels_dict, gamer_input))

    word = levels_dict[str(gamer_input)][3]
    attemts_number = levels_dict[str(gamer_input)][2]
    slovo = Slovo(word, attemts_number)
    input_number = 0

    print(slovo.show(False))
    while attemts_number != 0:
        input_letter = get_Level_or_Letter(False)
        input_number += 1

        new_word, attemts_number = slovo.game_mechanics(False, input_letter)
        print(f'{new_word} \nAttempts left: {attemts_number}')

        if new_word == slovo.show(True):
            print(f'\nCongratulations\nYou win\nYou completed the game in {input_number} turns')
            break
    if attemts_number == 0:  
        print('\nGame over')

In [ ]:
levels_dict = {'1': ['Easy', 5, 9, 'apple'], '2': ['Normal', 7, 7, 'shelter'], '3': ['Hard', 9, 7, 'binominal']}
game(levels_dict)